In [1]:
from langchain_groq import ChatGroq

In [2]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama-3.1-70b-versatile"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped onto the lunar surface on July 20, 1969, as part of the Apollo 11 mission.


In [3]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-42926?from=job%20search%20funnel")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


Apply for Software Engineer

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsTechnologySoftware EngineerBeaverton, OregonBecome a Part of the NIKE, Inc. TeamNIKE, Inc. does more than out

In [5]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
res.content

'```json\n{\n  "role": "Software Engineer",\n  "experience": "24 months of experience in the job offered or a computer-related occupation",\n  "skills": [\n    "Cognos analytics",\n    "Planning analytics and TM1",\n    "Teradata",\n    "Snowflake",\n    "Databricks",\n    "Oracle",\n    "SQL Server",\n    "AWS",\n    "Azure",\n    "ETL Tools",\n    "Scripting using Linux",\n    "Jenkins",\n    "AutoSys",\n    "Large scale systems and applications"\n  ],\n  "description": "Develop, code, configure, and test automations, programs, and solutions independently with minimal supervision to meet the process automation platform needs. Collaborate with cross-functional teams, including software engineers, architects, and product managers across the domains to understand the requirements and develop scalable platform capabilities with observability and high availability. Build pipelines to deploy orchestration services and automations in AWS and Azure. Define and implement the key metrics to ev

In [6]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Software Engineer',
 'experience': '24 months of experience in the job offered or a computer-related occupation',
 'skills': ['Cognos analytics',
  'Planning analytics and TM1',
  'Teradata',
  'Snowflake',
  'Databricks',
  'Oracle',
  'SQL Server',
  'AWS',
  'Azure',
  'ETL Tools',
  'Scripting using Linux',
  'Jenkins',
  'AutoSys',
  'Large scale systems and applications'],
 'description': 'Develop, code, configure, and test automations, programs, and solutions independently with minimal supervision to meet the process automation platform needs. Collaborate with cross-functional teams, including software engineers, architects, and product managers across the domains to understand the requirements and develop scalable platform capabilities with observability and high availability. Build pipelines to deploy orchestration services and automations in AWS and Azure. Define and implement the key metrics to evaluate the platform usage and provide useful insights to customers us

In [7]:
type(json_res)

dict

In [8]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [9]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [13]:
job = json_res
job['skills']

['Cognos analytics',
 'Planning analytics and TM1',
 'Teradata',
 'Snowflake',
 'Databricks',
 'Oracle',
 'SQL Server',
 'AWS',
 'Azure',
 'ETL Tools',
 'Scripting using Linux',
 'Jenkins',
 'AutoSys',
 'Large scale systems and applications']

In [14]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/flutter-portfolio'},
  {'links': 'https://example.com/react-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/flutter-portfolio'}],
 [{'links': 'https://example.com/android-tv-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/android-tv-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/flutter-portfolio'},
  {'links': 'https://example.com/kotlin-android-portfolio'}],
 [{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/angular-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/xamarin-portfolio'},
  {'links': 'https://example.com/angular-portfolio'}],
 [{'lin

In [15]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Alex, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Software Engineering Solutions for Your Business Needs

Dear [Client],

I came across your job description for a Software Engineer and was impressed by the scope of the role. As a Business Development Executive at AtliQ, I'd like to introduce you to our company's capabilities in delivering top-notch software engineering solutions that align with your requirements.

Our team of experts has extensive experience in developing, coding, configuring, and testing automations, programs, and solutions that meet the process automation platform needs. We have a proven track record of collaborating with cross-functional teams to understand requirements and develop scalable platform capabilities with observability and high availability.

Our expertise in the following areas aligns with your job requirements:

* Cloud platforms: AWS and Azure
* Data analytics: Teradata, Snowflake, Databricks, Oracle, and SQL Server
* ETL Tools and Scripting using Linux
* Automation tools: Jenkins and